In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/MyDrive/Tomato Plant'

/content/gdrive/MyDrive/Tomato Plant


In [3]:
import os
import cv2
import numpy as np

data_dir = "tomato/train/"
classes = ["healthy", "diseased"]
img_size = (224, 224)

data = []

for cls in classes:
    folder_path = os.path.join(data_dir, cls)
    label = classes.index(cls)
    
    for img_path in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)
        img = img / 255.0
        data.append([img, label])
        
np.random.shuffle(data)

X = []
y = []

for feature, label in data:
    X.append(feature)
    y.append(label)
    
X = np.array(X)
y = np.array(y)


In [4]:
# The preprocessing steps were already done during data collection.
# Ensure that the images are in the format (batch_size, height, width, channels)
X = np.reshape(X, (X.shape[0], img_size[0], img_size[1], 3))


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
from collections import deque

class DQNAgent:
    def __init__(self, state_size, action_size, num_models=5, memory_size=10000):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=memory_size)  # Using deque as a circular buffer for experience replay
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 32
        self.target_update_freq = 1000
        self.steps = 0
        self.num_models = num_models
        self.models = [self.build_model() for _ in range(num_models)]
        self.target_models = [self.build_model() for _ in range(num_models)]
        self.update_target_models()  # Update target models to sync with initial models
        
    def build_model(self):
        inputs = layers.Input(shape=self.state_size)
        x = layers.Conv2D(32, (3, 3), activation="relu")(inputs)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Conv2D(64, (3, 3), activation="relu")(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Conv2D(128, (3, 3), activation="relu")(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Flatten()(x)
        x = layers.Dense(256, activation="relu")(x)
        x = layers.Dense(128, activation="relu")(x)
        x = layers.Dense(64, activation="relu")(x)
        outputs = layers.Dense(self.action_size, activation="softmax")(x)
        model = models.Model(inputs=inputs, outputs=outputs)
        model.compile(loss="mse", optimizer=optimizers.Adam(learning_rate=0.01))
        return model
        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        else:
            q_values = np.zeros((self.num_models, self.action_size))
            for i in range(self.num_models):
                q_values[i] = self.models[i].predict(state)[0]
            return np.argmax(np.mean(q_values, axis=0))
        
    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        minibatch = np.random.choice(np.arange(len(self.memory)), size=self.batch_size, replace=False)
        minibatch = [self.memory[i] for i in minibatch]

        for state, action, reward, next_state, done in minibatch:
            for i in range(self.num_models):
                target = self.target_models[i].predict(state)
                if done:
                    target[0][action] = reward
                else:
                    Q_future = max(self.target_models[i].predict(next_state)[0])
                    target[0][action] = reward + Q_future * self.gamma
                self.models[i].fit(state, target, epochs=1, verbose=0)
                
        self.steps += 1
        if self.steps % self.target_update_freq == 0:
            self.update_target_models()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def update_target_models(self):
        for i in range(self.num_models):
            self.target_models[i].set_weights(self.models[i].get_weights())

In [6]:
import tensorflow as tf

# Force TensorFlow to use the GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


1 Physical GPUs, 1 Logical GPUs


In [ ]:
state_size = (img_size[0], img_size[1], 3)
action_size = len(classes)

agent = DQNAgent(state_size, action_size)

num_episodes = 50
steps_per_episode = X.shape[0] // num_episodes

for episode in range(num_episodes):
    for step in range(steps_per_episode):
        idx = episode * steps_per_episode + step
        state = X[idx:idx+1]
        action = agent.act(state)
        next_state = state
        reward = y[idx]
        done = True if step == steps_per_episode - 1 else False
        agent.remember(state, action, reward, next_state, done)
        agent.replay()

    print("Episode: {}/{}, Steps: {}".format(episode + 1, num_episodes, steps_per_episode))

Episode: 1/50, Steps: 4
Episode: 2/50, Steps: 4
Episode: 3/50, Steps: 4
Episode: 4/50, Steps: 4
Episode: 5/50, Steps: 4
Episode: 6/50, Steps: 4
Episode: 7/50, Steps: 4
1/1 [==============================] - 0s 85ms/step


1/1 [==============================] - 0s 78ms/step


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 21ms/step
Episode: 9/50, Steps: 4
1/1 [==============================] - 0s 21ms/step
Episode: 10/50, Steps: 4
1/1 [==============================] - 0s 20ms/step
Episode: 11/50, Steps: 4
1/1 [==============================] - 0s 20ms/step
Episode: 12/50, Steps: 4
1/1 [==============================] - 0s 81ms/step


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 18ms/step
Episode: 45/50, Steps: 4
1/1 [==============================] - 0s 19ms/step
Episode: 46/50, Steps: 4
1/1 [==============================] - 0s 21ms/step
Episode: 47/50, Steps: 4
1/1 [==============================] - 0s 20ms/step
Episode: 48/50, Steps: 4
1/1 [==============================] - 16s 16s/step


In [ ]:
test_data_dir = "tomato/val/"

test_data = []

for cls in classes:
    folder_path = os.path.join(test_data_dir, cls)
    label = classes.index(cls)
    
    for img_path in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)
        img = img / 255.0
        test_data.append([img, label])
        
np.random.shuffle(test_data)

X_test = []
y_test = []

for feature, label in test_data:
    X_test.append(feature)
    y_test.append(label)
    
X_test = np.array(X_test)
y_test = np.array(y_test)

X_test = np.reshape(X_test, (X_test.shape[0], img_size[0], img_size[1], 3))

correct = 0

for i in range(X_test.shape[0]):
    state = X_test[i:i+1]
    action = agent.act(state)
    if action == y_test[i]:
        correct += 1
        
accuracy = correct / X_test.shape[0]

print("Accuracy:", accuracy)


In [ ]:
# Load a single test image
img = cv2.imread("tomato/val/healthy/01c1da17-8d9f-4d69-8a1e-58d37453d3c3___RS_HL 9641.JPG")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, img_size)
img = img / 255.0

# Reshape the image to match the input shape of the DQN agent
state = np.reshape(img, (1, img_size[0], img_size[1], 3))

# Use the act() method of the DQN agent to predict the class of the image
action = agent.act(state)

# Map the predicted action to the corresponding class label
predicted_class = classes[action]

print("Predicted class:", predicted_class)

# Load a single test image
img = cv2.imread("tomato/val/diseased/bacterial_spot (26).JPG")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, img_size)
img = img / 255.0

# Reshape the image to match the input shape of the DQN agent
state = np.reshape(img, (1, img_size[0], img_size[1], 3))

# Use the act() method of the DQN agent to predict the class of the image
action = agent.act(state)

# Map the predicted action to the corresponding class label
predicted_class = classes[action]

print("Predicted class:", predicted_class)
